# Iris Classification

In [1]:
# 导入
import sys

sys.path.append("E:/dataFiles/github/MFlow")

In [2]:
# 数据生成
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# TODO: 不使用pandas以及sklearn

# 数据读取，并去掉第一列
data = pd.read_csv("E:/dataFiles/github/MFlow/assets/Iris.csv").drop("Id", axis=1)
# 打乱样本
data = data.sample(len(data), replace=False)
xs = data[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]].values
# 将字符串标签替换为数字
le = LabelEncoder()
label = le.fit_transform(data["Species"])
# 转换为one-hot
ohe = OneHotEncoder(sparse=False)
ys = ohe.fit_transform(label.reshape(-1, 1))

print(xs.shape, ys.shape)
print(xs[0], ys[0])

(150, 4) (150, 3)
[4.8 3.1 1.6 0.2] [1. 0. 0.]


In [3]:
# 训练
from mflow import core, ops, opts

# 超参数
lr = 0.02
epoch = 30
batch_size = 16

with core.NameScope("IrisClassification"):
    # 初始化变量
    x = core.Variable(size=(4, 1), trainable=False)
    y = core.Variable(size=(3, 1), trainable=False)
    w = core.Variable(size=(3, 4), trainable=True)
    b = core.Variable(size=(3, 1), trainable=True)
    # 模型定义
    pred = ops.Add(ops.MatMul(w, x), b)
    predicter = ops.SoftMax(pred)
    loss = ops.loss.CrossEntropyWithSoftMax(pred, y)
    adam = opts.Adam(core.DefaultGraph, loss, lr)
    # 开始训练
    for ep in range(1, epoch + 1):
        bs_idx = 0  # 批次计数
        # 这是一个epoch的过程
        for i, (feat, one_hot) in enumerate(zip(xs, ys)):
            x.setValue(np.mat(feat).T)
            y.setValue(np.mat(one_hot).T)
            adam.step()
            bs_idx += 1
            if bs_idx == batch_size:
                adam.update()
                bs_idx = 0
        # 一个epoch完成后进行评估
        preds = []
        for feat in xs:
            x.setValue(np.mat(feat).T)
            predicter.forward()
            preds.append(predicter.value.A.ravel())  # 结果
        preds = np.array(preds).argmax(axis=1)
        acc = (label == preds).astype("uint8").sum() / len(xs)
        print("Epoch: {:d}, acc: {:.3f}.".format(ep, acc))

Epoch: 1, acc: 0.567.
Epoch: 2, acc: 0.667.
Epoch: 3, acc: 0.707.
Epoch: 4, acc: 0.840.
Epoch: 5, acc: 0.820.
Epoch: 6, acc: 0.907.
Epoch: 7, acc: 0.900.
Epoch: 8, acc: 0.933.
Epoch: 9, acc: 0.947.
Epoch: 10, acc: 0.953.
Epoch: 11, acc: 0.953.
Epoch: 12, acc: 0.953.
Epoch: 13, acc: 0.953.
Epoch: 14, acc: 0.953.
Epoch: 15, acc: 0.953.
Epoch: 16, acc: 0.960.
Epoch: 17, acc: 0.967.
Epoch: 18, acc: 0.967.
Epoch: 19, acc: 0.967.
Epoch: 20, acc: 0.967.
Epoch: 21, acc: 0.967.
Epoch: 22, acc: 0.973.
Epoch: 23, acc: 0.973.
Epoch: 24, acc: 0.973.
Epoch: 25, acc: 0.973.
Epoch: 26, acc: 0.973.
Epoch: 27, acc: 0.980.
Epoch: 28, acc: 0.987.
Epoch: 29, acc: 0.987.
Epoch: 30, acc: 0.980.
